In [ ]:
import numpy as np
import pandas as pd

def process_stock_data(stock_data, columns_to_process):
    # 将 trade_month 转换为 datetime
    stock_data["trade_month"] = pd.to_datetime(stock_data["trade_month"])
    
    # 提取季度末数据
    stock_data["quarter"] = stock_data["trade_month"].dt.to_period("Q")
    
    # 创建一个映射：季度 -> 每列的第一个非空值（如 ZSCORE）
    for column in columns_to_process:
        # We first check if the column exists in the dataset
        if column not in stock_data.columns:
            print(f"Warning: Column '{column}' not found in data!")
            continue
        
        # 对每个 stock_code 进行分组处理
        for stock_code, group_data in stock_data.groupby("stock_code"):
            # Drop rows with missing values in the column and set 'trade_month' as the index
            column_data = group_data.dropna(subset=[column]).set_index("trade_month")
            
            # Create a mapping for the first non-null value of the column in each quarter
            quarter_column_map = column_data.groupby("quarter")[column].first()
            
            # Map the quarterly values to the group_data frame
            stock_data.loc[group_data.index, column] = group_data["quarter"].map(quarter_column_map)
            
            # Perform forward filling for any missing values in the column
            stock_data[column] = stock_data[column].fillna(method="ffill")
    
    return stock_data

# Example usage
# columns_to_process = ["ZSCORE", "factor2", "factor3"]  # Add other columns you want to process
# processed_data = process_stock_data(stock_data, columns_to_process)
path = r"C:\Users\Fisher Man\OneDrive\Desktop\Work Sheet\Py\Some Projects\Fin-Econ\Project\FF3\处理后数据2.0.dta"
panel_data = pd.read_stata(path)
panel_data.info()
labels = ["证券代码","交易月份","excess_return",'市场风险溢价因子流通市值加权','市值因子流通市值加权','账面市值比因子流通市值加权',"每股企业自由现金流量","EM"]
panel_data = panel_data[labels]
panel_data.rename(columns={
    '证券代码': 'stock_code',
    '交易月份': 'trade_month',
    '每股企业自由现金流量': 'free_cash_flow',
    # '市净率PB': 'pb_ratio',
    # 'return1': 'return',
    # 'RiskPremium': 'mkt',
    '市场风险溢价因子流通市值加权':'mkt',
    '市值因子流通市值加权':'smb',
    '账面市值比因子流通市值加权':'hml',
    'EM': 'em'
    # 'SMB': 'smb',
    # 'HML': 'hml'
}, inplace=True)
indicators = ['free_cash_flow','em']
panel_data= process_stock_data(panel_data,indicators)
panel_data.to_csv('panel_data.csv',index=False)
data = pd.read_csv('panel_data.csv')
# data.astype("float64")
print(data.info())
print(data.head())
data = data.dropna(subset='excess_return')  # Drop missing values
data.isnull().sum()


In [ ]:
data = pd.read_csv('panel_data.csv')
# data.astype("float64")
print(data.info())
print(data.head())
data = data.dropna(subset='excess_return')  # Drop missing values
data.isnull().sum()

In [ ]:
import numpy as np
import pandas as pd

# 定义排序因子
factor2 = indicators[0]  # 排序因子
factor1 = indicators[1]  # 排序因子

# 打印数据形状
print(data.shape)

# 初始化结果容器
grouped_data = []

# 遍历每个交易月
for trading_month, monthly_data in data.groupby('trade_month'):
    # 定义排序组数
    num_groups = 5

    # 按 factor1 排序并分组
    monthly_data = monthly_data.sort_values(by=factor1).reset_index(drop=True)
    monthly_data['factor1_quantile'] = (np.floor(monthly_data.index / (len(monthly_data) / num_groups)) + 1).astype(int)

    # 对每个 factor1 分组，进一步按 factor2 排序
    for q1 in range(1, num_groups + 1):
        group1 = monthly_data[monthly_data['factor1_quantile'] == q1]
        
        # 检查当前 group1 是否为空，避免后续操作出错
        if group1.empty:
            continue
        
        # 按 factor2 排序并创建量化组
        group1 = group1.sort_values(by=factor2).reset_index(drop=True)
        group1['factor2_quantile'] = (np.floor(group1.index / (len(group1) / num_groups)) + 1).astype(int)

        # 对于每个 factor2 分组，计算均值
        for q2 in range(1, num_groups + 1):
            group2 = group1[group1['factor2_quantile'] == q2]
            print(f"Trading Month: {trading_month}, Factor1 Quantile: {q1}, Factor2 Quantile: {q2}, shape: {group2.shape}")
            
            # 计算均值
            if not group2.empty:  # 仅在非空时计算均值
                y = group2['excess_return'].mean()
                mkt = group2['mkt'].mean()
                smb = group2['smb'].mean()
                hml = group2['hml'].mean()
                
                grouped_data.append(
                    {
                        'trading_month': trading_month,
                        'factor1_quantile': q1,
                        'factor2_quantile': q2,  # 添加 factor2_quantile 信息
                        'avg_excess_return': y,
                        'mkt': mkt,
                        'smb': smb,
                        'hml': hml
                    }
                )
    
    # # 计算 (1,1) 和 (5,5) 的均值差异
    # first_quantile = monthly_data[(monthly_data['factor1_quantile'] == 1) & (monthly_data['factor2_quantile'] == 1)]
    # last_quantile = monthly_data[(monthly_data['factor1_quantile'] == 5) & (monthly_data['factor2_quantile'] == 5)]
    
    # # 计算均值差异
    # first_mean = first_quantile[['excess_return', 'mkt', 'smb', 'hml']].mean()
    # last_mean = last_quantile[['excess_return', 'mkt', 'smb', 'hml']].mean()
    
    # diff = last_mean - first_mean
    
    # # 将差异结果添加到 grouped_data
    # grouped_data.append({
    #     'trading_month': trading_month,
    #     'factor1_quantile': 26,  # 使用 26 表示差异组
    #     'factor2_quantile': 26,  # 对于这个特殊组也可以标记
    #     'avg_excess_return': diff['excess_return'],
    #     'mkt': diff['mkt'],
    #     'smb': diff['smb'],
    #     'hml': diff['hml']
    # })

# 转换为 DataFrame 并保存
grouped_data_df = pd.DataFrame(grouped_data)
grouped_data_df.to_csv('reversed_grouped_data.csv', index=False)


In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
grouped_data_df = pd.read_csv("reversed_grouped_data.csv")
# 定义回归函数
def regress_ff3(group):
    y = group['avg_excess_return']
    X = group[['mkt', 'smb', 'hml']]
    X = sm.add_constant(X)  # 增加常数项
    model = sm.OLS(y, X).fit()
    return {
        "factor1_quantile": group['factor1_quantile'].iloc[0],
        "factor2_quantile": group['factor2_quantile'].iloc[0],
        't-stat (alpha)': model.tvalues.get('const', np.nan),
        'p-value (alpha)': model.pvalues.get('const', np.nan),
        "alpha": model.params.get('const', np.nan),
        'p-value (mkt)': model.pvalues.get('mkt', np.nan),
        "mkt_coef": model.params.get('mkt', np.nan),
        'p-value (smb)': model.pvalues.get('smb', np.nan),
        "smb_coef": model.params.get('smb', np.nan),
        'p-value (hml)': model.pvalues.get('hml', np.nan),
        "hml_coef": model.params.get('hml', np.nan),
        "r_squared": model.rsquared
    }

# 应用双因子分组的回归分析
results = (
    grouped_data_df
    .groupby(['factor1_quantile', 'factor2_quantile'])
    .apply(regress_ff3)
    .apply(pd.Series)
)

# 打印或保存结果
# print(results)
results.to_csv("reverse_sort_double_factor_regression_results.csv", index=False)


In [ ]:
correlation = data[[factor1, factor2]].corr()
print(correlation)
print(data[factor1].nunique(), data[factor2].nunique())
# print(monthly_data['factor1_quantile'].value_counts())
# print(monthly_data['factor2_quantile'].value_counts())
